In [58]:
import pandas as pd
import numpy as np
from collections import defaultdict

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk import pos_tag
from nltk.corpus import stopwords

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score

import seaborn as sns

In [59]:
np.random.seed(500)

In [60]:
dados=pd.read_csv(r"/home/roberto/Área de trabalho/NLP/dados/folha_app_ronda.csv",encoding='utf-8')
dados.head()

,data,titulo,url,conteudo,categoria
0,2020-09-01,"Após saída de Deltan, Aras articula encurtar d...",https://www1.folha.uol.com.br/poder/2020/09/ap...,Após a saída do procurador Deltan Dallagnol da...,Lava Jato
1,2020-09-02,STJ decide manter Witzel afastado por 6 meses ...,https://www1.folha.uol.com.br/poder/2020/09/st...,O STJ (Superior Tribunal de Justiça) decidiu n...,Casos corrupcao
2,2020-09-03,Escolha de vice e elo com Doria desafiam Bruno...,https://www1.folha.uol.com.br/poder/2020/09/es...,A poucos dias de o PSDB confirmar a candidatur...,Eleicoes
3,2020-09-03,"Sob resistências, Bolsonaro pretende indicar m...",https://www1.folha.uol.com.br/poder/2020/09/so...,Com a decisão do ministro José Múcio Monteiro ...,Politica Nacional
4,2020-09-03,"STF não vê clima para salvar Witzel, mas avali...",https://www1.folha.uol.com.br/poder/2020/09/st...,A avaliação da maioria dos ministros do STF (S...,Casos corrupcao


In [61]:
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
dados['titulo'] = [entry.lower() for entry in dados['titulo']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
dados['tokens']= [word_tokenize(entry) for entry in dados['titulo']]
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(dados['tokens']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('portuguese') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    dados.loc[index,'tokens_final'] = str(Final_words)

In [62]:
dados['categoria']=dados['categoria'].astype('str')
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(dados['tokens_final'],dados['categoria'],test_size=0.3,random_state=13)

Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)
print(Train_X)
print(Test_Y)
Tfidf_vect = TfidfVectorizer(max_features=5000)
fit=Tfidf_vect.fit(dados['tokens_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

70     ['medidas', 'kalil', 'pandemia', 'viram', 'alv...
63     ['autoridades', 'ignoram', 'pandemia', 'fazem'...
56     ['bolsonaro', 'recorre', 'contra', 'decisão', ...
102    ['supremo', 'resiste', 'jorge', 'oliveira', 'b...
81     ['afastar', 'aplicação', 'direito', 'internet'...
                             ...                        
126    ['fux', 'marca', 'presença', 'celso', 'julgame...
98     ['direita', 'conservadora', 'mira', 'capitais'...
16     ['fhc', 'faz', 'meaculpa', 'afirma', 'emenda',...
74     ['decisões', 'stf', 'últimos', 'ano', 'tomada'...
82     ['votos', 'assembleia', 'rio', 'autoriza', 'pr...
Name: tokens_final, Length: 95, dtype: object
[1 1 3 6 2 6 1 1 5 1 1 6 1 1 2 0 6 1 4 6 2 1 1 7 6 1 0 1 1 6 4 1 0 1 3 6 1
 7 4 2 1 1]


In [63]:
#fit.get_feature_names()
#print(Tfidf_vect.vocabulary_)

In [64]:
#print(Test_X_Tfidf)

In [65]:
SVM = svm.SVC(C=10, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
predictions_SVM = SVM.predict(Test_X_Tfidf)
#print("Predictions -> ",predictions_SVM)
#previsoes=pd.DataFrame(data={'titulo':Test_X,'categoria_prevista':Encoder.inverse_transform(predictions_SVM), 'categoria_original':Encoder.inverse_transform(Test_Y)})
#previsoes.to_csv('previsoes.csv')
for row_index, (input, prediction, label) in enumerate(zip (Test_X, predictions_SVM, Test_Y)):
  if prediction != label:
    print('Linha', row_index, 'foi classificada como ', prediction, 'mas devia ser ', label)

Linha 2 foi classificada como  0 mas devia ser  3
Linha 4 foi classificada como  1 mas devia ser  2
Linha 8 foi classificada como  1 mas devia ser  5
Linha 16 foi classificada como  1 mas devia ser  6
Linha 18 foi classificada como  1 mas devia ser  4
Linha 19 foi classificada como  1 mas devia ser  6
Linha 20 foi classificada como  3 mas devia ser  2
Linha 21 foi classificada como  6 mas devia ser  1
Linha 23 foi classificada como  0 mas devia ser  7
Linha 24 foi classificada como  1 mas devia ser  6
Linha 26 foi classificada como  1 mas devia ser  0
Linha 29 foi classificada como  1 mas devia ser  6
Linha 30 foi classificada como  1 mas devia ser  4
Linha 35 foi classificada como  1 mas devia ser  6
Linha 37 foi classificada como  1 mas devia ser  7
Linha 38 foi classificada como  1 mas devia ser  4


In [66]:
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  61.904761904761905
